In [1]:
from enum import Enum
from scipy.optimize import minimize, Bounds, differential_evolution
import numpy as np
import matplotlib.pyplot as plt
from typing import List

In [2]:
PRICES = np.array([1, 10, 10, 1, 10, 10, 1, 1, 1, 1, 10, 10, 100, 1, 1]).repeat(2)
COOLING_PER_HOUR = 0.5
HEATING_PER_HOUR = 2.0

START_TEMP = 20.0
MIN_TEMP = 18.0
MAX_TEMP = 22.0
PENALTY_SCALE = 1_000.0
BINARY_REG_SCALE = 1e-2  # small, encourages 0/1 while keeping smoothness

def objective(actions: np.ndarray) -> float:
    temp = START_TEMP
    cost = 0
    for i, a in enumerate(actions): 
        action = "ON" if a > 0.5 else "OFF"
        temp -= COOLING_PER_HOUR
        if action == "ON":
            temp += HEATING_PER_HOUR
        cost += PRICES[i] 

        if temp < MIN_TEMP:
            penalty = PENALTY_SCALE * (MIN_TEMP - temp) ** 3
            cost += penalty
        if temp > MAX_TEMP:
            penalty = PENALTY_SCALE * (temp - MAX_TEMP) ** 3
            cost += penalty

    return cost


x0 = np.full(len(PRICES), 0.5)  # nontrivial start to avoid flat region
bounds = [(0, 1)] * len(PRICES)
result = differential_evolution(objective, bounds=bounds, maxiter=200, polish=False)
# result = minimize(objective, x0, bounds=bounds, method="SLSQP")

temp = START_TEMP
print("Actions | Temps | Price")
for i, x in enumerate(result.x):
    action = "ON" if x > 0.5 else "OFF"
    temp -= COOLING_PER_HOUR
    if action == "ON":
        temp += HEATING_PER_HOUR
    print(f"{action:7s} | {temp:.2f} | {PRICES[i]:.2f}")

print("Total cost: ", result.fun)
print("Best action: ", result.x[0])



Actions | Temps | Price
OFF     | 19.50 | 1.00
ON      | 21.00 | 1.00
OFF     | 20.50 | 10.00
ON      | 22.00 | 10.00
OFF     | 21.50 | 10.00
OFF     | 21.00 | 10.00
OFF     | 20.50 | 1.00
OFF     | 20.00 | 1.00
OFF     | 19.50 | 10.00
OFF     | 19.00 | 10.00
OFF     | 18.50 | 10.00
ON      | 20.00 | 10.00
OFF     | 19.50 | 1.00
OFF     | 19.00 | 1.00
OFF     | 18.50 | 1.00
ON      | 20.00 | 1.00
OFF     | 19.50 | 1.00
OFF     | 19.00 | 1.00
OFF     | 18.50 | 1.00
OFF     | 18.00 | 1.00
ON      | 19.50 | 10.00
ON      | 21.00 | 10.00
OFF     | 20.50 | 10.00
OFF     | 20.00 | 10.00
OFF     | 19.50 | 100.00
OFF     | 19.00 | 100.00
OFF     | 18.50 | 1.00
ON      | 20.00 | 1.00
OFF     | 19.50 | 1.00
ON      | 21.00 | 1.00
Total cost:  336.0
Best action:  0.13234084467746554


In [3]:
import numpy as np
from scipy.optimize import differential_evolution  # if you still want a heuristic

PRICES = np.array([1, 10, 10, 1, 10, 10, 1, 1, 1, 1, 10, 10, 100, 1, 1]).repeat(5)
COOLING_PER_HOUR = 0.5
HEATING_PER_HOUR = 2.0

START_TEMP = 20.0
MIN_TEMP = 18.0
MAX_TEMP = 22.0
PENALTY_SCALE = 1_000.0

T = len(PRICES)
idx = np.arange(1, T+1)

def penalties(temp):
    # cubic soft constraints
    low = np.maximum(0.0, MIN_TEMP - temp)
    high = np.maximum(0.0, temp - MAX_TEMP)
    return PENALTY_SCALE * (low**3 + high**3)

def simulate_temps(actions):
    # actions \in [0,1] (or {0,1} if you later round)
    # temp at step i *after* applying action i: 
    # START - i*cool + (HEAT)*cumsum(actions)[i]
    heat_cum = np.cumsum(actions)
    temps = START_TEMP - COOLING_PER_HOUR*idx + HEATING_PER_HOUR*heat_cum
    return temps

def objective(actions):
    temps = simulate_temps(actions)
    energy_cost = (PRICES * actions).sum()   # pay only when heating
    return energy_cost + penalties(temps).sum()

# If you insist on DE, vectorization alone makes it much faster:
bounds = [(0.0, 1.0)] * T
result = differential_evolution(objective, bounds=bounds, maxiter=500, polish=False)

temp = START_TEMP
print("Actions | Temps | Price")
for i, x in enumerate(result.x):
    action = "ON" if x > 0.5 else "OFF"
    temp -= COOLING_PER_HOUR
    if action == "ON":
        temp += HEATING_PER_HOUR
    print(f"{action:7s} | {temp:.2f} | {PRICES[i]:.2f}")

print("Total cost: ", result.fun)
print("Best action: ", result.x[0])


Actions | Temps | Price
ON      | 21.50 | 1.00
OFF     | 21.00 | 1.00
ON      | 22.50 | 1.00
OFF     | 22.00 | 1.00
OFF     | 21.50 | 1.00
OFF     | 21.00 | 10.00
OFF     | 20.50 | 10.00
OFF     | 20.00 | 10.00
OFF     | 19.50 | 10.00
OFF     | 19.00 | 10.00
OFF     | 18.50 | 10.00
OFF     | 18.00 | 10.00
OFF     | 17.50 | 10.00
OFF     | 17.00 | 10.00
OFF     | 16.50 | 10.00
OFF     | 16.00 | 1.00
OFF     | 15.50 | 1.00
OFF     | 15.00 | 1.00
ON      | 16.50 | 1.00
OFF     | 16.00 | 1.00
OFF     | 15.50 | 10.00
OFF     | 15.00 | 10.00
OFF     | 14.50 | 10.00
OFF     | 14.00 | 10.00
OFF     | 13.50 | 10.00
OFF     | 13.00 | 10.00
OFF     | 12.50 | 10.00
OFF     | 12.00 | 10.00
OFF     | 11.50 | 10.00
OFF     | 11.00 | 10.00
OFF     | 10.50 | 1.00
OFF     | 10.00 | 1.00
OFF     | 9.50 | 1.00
OFF     | 9.00 | 1.00
OFF     | 8.50 | 1.00
OFF     | 8.00 | 1.00
OFF     | 7.50 | 1.00
OFF     | 7.00 | 1.00
ON      | 8.50 | 1.00
ON      | 10.00 | 1.00
OFF     | 9.50 | 1.00
OFF     | 9.00 | 1.00

In [4]:
import numpy as np

PRICES = np.array([1, 10, 10, 1, 10, 10, 1, 1, 1, 1, 10, 10, 100, 1, 1]).repeat(10)
COOLING_PER_HOUR = 0.5
HEATING_PER_HOUR = 2.0

START_TEMP = 20.0
MIN_TEMP = 18.0
MAX_TEMP = 22.0
PENALTY_SCALE = 1_000.0

T = len(PRICES)

# Discretize temperature; ~0.1°C is usually plenty
step = 0.1
# Pad the range a bit to allow overshoot without truncation
grid_min, grid_max = MIN_TEMP - 3.0, MAX_TEMP + 3.0
temps_grid = np.round(np.arange(grid_min, grid_max + step, step), 10)
S = len(temps_grid)

def penalty(temp):
    if temp < MIN_TEMP:
        return PENALTY_SCALE * (MIN_TEMP - temp)**3
    if temp > MAX_TEMP:
        return PENALTY_SCALE * (temp - MAX_TEMP)**3
    return 0.0

# Precompute penalty for every grid temp
penalty_lut = np.array([penalty(t) for t in temps_grid])

# Transition: given state index s and action a in {0,1}
def next_state_idx(s, a):
    next_temp = temps_grid[s] - COOLING_PER_HOUR + a * HEATING_PER_HOUR
    # snap to nearest grid index
    j = int(np.clip(np.round((next_temp - grid_min)/step), 0, S-1))
    return j

# Build transition tables (vectorized over states)
ns0 = np.array([next_state_idx(s, 0) for s in range(S)], dtype=np.int32)
ns1 = np.array([next_state_idx(s, 1) for s in range(S)], dtype=np.int32)

# DP arrays
INF = 1e30
V = np.full((T+1, S), 0.0)           # value-to-go
pi = np.zeros((T, S), dtype=np.uint8) # optimal action

# Backward pass
for t in range(T-1, -1, -1):
    # Cost if a=0
    c0 = penalty_lut[ns0] + V[t+1, ns0] + 0.0            # no energy cost if OFF
    # Cost if a=1
    c1 = penalty_lut[ns1] + V[t+1, ns1] + PRICES[t]      # pay price when ON
    better_is_one = c1 < c0
    V[t] = np.where(better_is_one, c1, c0)
    pi[t] = better_is_one.astype(np.uint8)

# Recover start-state index
s0 = int(np.clip(np.round((START_TEMP - grid_min)/step), 0, S-1))
best_cost = V[0, s0]

# Roll forward to extract actions and temps
actions = np.zeros(T, dtype=np.uint8)
temps = np.zeros(T)
s = s0
for t in range(T):
    a = pi[t, s]
    actions[t] = a
    s = ns1[s] if a else ns0[s]
    temps[t] = temps_grid[s]

print("Optimal total cost:", float(best_cost))
print("ON hours:", actions.sum(), "of", T)
# actions is 0/1 optimal schedule; temps are the resulting temps after each hour

Optimal total cost: 244.0
ON hours: 37 of 150


Planned trajectory (post-step state each hour)
t  Action  Temp(°C)  Price  Energy  Penalty
-- ------  --------  -----  ------  -------
 0  OFF       19.50      1       0      0.0
 1  OFF       19.00      1       0      0.0
 2  OFF       18.50      1       0      0.0
 3  OFF       18.00      1       0      0.0
 4  ON        19.50      1       1      0.0
 5  OFF       19.00      1       0      0.0
 6  OFF       18.50      1       0      0.0
 7  OFF       18.00      1       0      0.0
 8  ON        19.50      1       1      0.0
 9  ON        21.00      1       1      0.0
10  OFF       20.50     10       0      0.0
11  OFF       20.00     10       0      0.0
12  OFF       19.50     10       0      0.0
13  OFF       19.00     10       0      0.0
14  OFF       18.50     10       0      0.0
15  OFF       18.00     10       0      0.0
16  ON        19.50     10      10      0.0
17  OFF       19.00     10       0      0.0
18  OFF       18.50     10       0      0.0
19  OFF       18.00     10   